In [1]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))
sys.path.append(project_path)
print(f"Project Path: {project_path}")

Project Path: c:\Users\dalej\Documents\_Coding\DragonRegen


## Testing Kafka Agent

In [2]:
from src.AIGuardian.Tasks.KafkaAgent import KafkaAgent

agent = KafkaAgent(end_processing=20)
agent.receive_messages()

Agent Loop Count: 0 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 1 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 2 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 3 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 4 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 5 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 6 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 7 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 8 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 9 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 10 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 11 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 12 and last consumed time: 2025-05-21 21:20:44.222629
Agent Loop Count: 13 and last consumed time: 2025-05-21 21:20:44.222629
Ag

KeyboardInterrupt: 

In [3]:
print(agent.waiting_tasks)
print(agent.processed_tasks)

[<src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000002625A48F8D0>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000002625A7E7250>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000002625A7E7450>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000002625A7DC5D0>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000002625A48D890>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000002625A48E790>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000002625A7DC910>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x0000026258D4A050>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000002625A8F2650>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000002625A8D6390>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000002625A8A6850>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x